In [ ]:
#単一の CW レーザーを照射 ver.
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # Laguerre多項式のために追加

core_num = 20 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "simdata" # 保存パスはご自身の環境に合わせて変更してください

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials # Laguerre多項式のために追加

    #parameters
    Tave = 298 #room temperature
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 10e-3 #incident laser power(mW)
    # ls_spot_x = 0 #x-position of the center of the laser spot (LGビームは中心が暗いコアなので、ls_spot_x,yは直接使用しない)
    # ls_spot_y = 0 #y-position of the center of the laser spot
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens (LGビームのw0_beamを直接設定するため、FWHMやg_wは使用しない)
    # FWHM = 1.22 * w_l / NA #full width at half maximum of the gaussian beam
    # g_w = FWHM * sqrt(log(2) / 2) #Gaussian width at half maximum of the gaussian beam
    p_num = 2 #number of particles
    t = 60 #calculate time
    n = 10 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    # LGビーム固有のパラメータ
    p_index = 0 # 半径方向モード数 (p >= 0)
    l_index = 1 # 方位角モード数 (l は整数, l!=0 でドーナツ型)
    w0_beam = 1.0e-6 # ビームウェスト w0 (m), 例: 1マイクロメートル。NAから計算することも可能だが、ここでは直接設定。

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    # dynamic viscosity of water
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    # variance of the BM
    function sigma(T)
        sigma = sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
        return sigma
    end
    # friction coefficient
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # generate random numbers
     function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end
    # --- LGビーム関連関数 (ユーザー提供のコードを参考に、勾配力計算用に調整) ---

    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function w_z(z, w0, zR)
        return w0 * sqrt(1 + (z/zR)^2)
    end

    """
        Laguerre_L(p::Int, l::Int, x::Real)

    SpecialPolynomials.jl を使用して、関連ラゲール多項式 L_p^(l)(x) を評価します。
    """
    function Laguerre_L(p::Int, l::Int, x::Real)
        if p < 0 || l < 0
            throw(ArgumentError("指数 p および l は非負の整数でなければなりません。"))
        end
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l)}, p) # [5]
        return L_poly(x)
    end

    # LGビームの強度プロファイル (正規化された形)
    # この関数は、ビームの空間的な強度分布の「形」を返します。
    # 絶対的な強度にするには、後でレーザーパワーでスケーリングする必要があります。
    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w_z(z, w0, zR)
        x_arg = 2 * r^2 / current_wz^2

        # r=0でl > 0の場合の特異点処理 (強度0) [5]
        if r == 0 && l!= 0
            return 0.0
        end

        radial_term_power = x_arg^abs(l)
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        # (w0/current_wz)^2 はビームの伝搬による強度変化を表す [5]
        return (w0/current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    # 粒子の分極率 (Rayleigh近似) [5]
    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2 # 媒質の誘電率 (mu_r = 1 と仮定)
        return 4 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2 * nm^2))
    end

    # LGビームによる光勾配力計算
    # 元の fot 関数を置き換える
    # 2Dシミュレーションのため、Fx, Fy のみを返す
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        # デカルト座標から円筒座標へ変換
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos) # atan2(y,x) がより堅牢だが、ここでは atan で十分

        # 粒子の分極率
        alpha_particle = particle_polarizability(r_p, np, nm)

        # 数値微分用の微小ステップ
        # 解析的微分が最も正確だが、実装の複雑さを考慮し、ここでは数値微分を使用 [5]
        dr_step = 1e-10 # 半径方向の微分ステップ

        # 現在位置での正規化された強度
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)

        # 半径方向の強度勾配 (dI/dr) を計算
        # rが非常に小さい場合 (特にr=0でl!=0の場合)、r+dr_stepが0になることを避ける
        r_plus_dr = r + dr_step
        if r_plus_dr < 0 # 負の値にならないように保護
            r_plus_dr = 0.0
        end
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        # レーザーパワーによる強度スケーリングファクター
        # LGビームの全パワーを考慮した正規化定数が必要だが、
        # 簡単のため、ガウスビームのピーク強度に相当するスケーリングを使用 [6]
        I_scaling_factor = ls_pow / (pi * w0_beam^2) # 例: ガウスビームのピーク強度に相当するスケーリング

        # 半径方向の勾配力 (F_r = alpha * dI/dr) [5]
        # 光勾配力は (1/2) * alpha * grad(|E|^2) で与えられる。
        # |E|^2 は強度 I に比例するため、F_grad は alpha * grad(I) に比例する。
        # ここで計算される dI_dr は正規化された強度に対する勾配なので、
        # 実際のパワーによるスケーリングファクターを乗じる。
        F_r = alpha_particle * dI_dr * I_scaling_factor

        # 円筒座標の力をデカルト座標に変換
        # 理想的なLGビームでは方位角方向の勾配力はゼロ (F_phi = 0) [5]
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        Fz = 0.0 # 2Dシミュレーションのため、軸方向の力は考慮しない

        return Fx, Fy, Fz
    end

    # generate displacements of BM
    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab.* xr
        deltayb = sigmab.* yr
        return deltaxb, deltayb
    end

    # calculate a particle behavior
    # LGビームパラメータを引数に追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)

        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームによる勾配力を計算
            Fx_grad, Fy_grad, Fz_grad = LG_Gradient_Force(x_i[i], y_i[i], 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # オイラー法で位置を更新
            # ブラウン運動の変位 + 光勾配力による変位
            x_i[i+1] = x_i[i] + xb[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end

    # define parallel calculations
    # LGビームパラメータを引数に追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx.* 1e6
            resulty = resulty.* 1e6
        end
        cd("$(save_path)")
        writedlm("x_LG_p
(l_idx)_(ls_pow)mW_
(r_p)nm.txt", resultx, "\t")
        writedlm("y_LG_p
(l_idx)_(ls_pow)mW_
(r_p)nm.txt", resulty, "\t")
    end

    #main
    # LGビームパラメータを引数に追加
    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx, l_idx, w_l)
        # generate displacements of Brrownian motion
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        # generate initial positionas
        # LGビームの中心は(0,0)なので、初期位置はビーム中心からの相対位置となる
        x0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        y0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# Main execution call
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())

In [3]:
#単一の CW レーザーを照射 ver.
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # Laguerre多項式のために追加

# --- ユーザーが変更するLGビームパラメーター ---
p_index = 1 # 半径方向モード数 (p >= 0)。ドーナツの輪の数に関係
l_index = 2 # 方位角モード数 (l は整数, l!=0 でドーナツ型)。軌道角運動量に関係
# ------------------------------------------

core_num = 20 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "C:\\Users\\setol\\Documents\\sim_data" # 保存パスはご自身の環境に合わせて変更してください

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials # Laguerre多項式のために追加

    #parameters
    Tave = 298 #room temperature
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 10e-3 #incident laser power(mW)
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 2 #number of particles
    t = 60 #calculate time
    n = 10 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value

    # LGビーム固有のパラメータ (メインスクリプトから引き継がれるが、ここでは関数の引数として受け取る)
    # p_index, l_index は main 関数を通じて渡される

    w0_beam = 1.0e-6 # ビームウェスト w0 (m), 例: 1マイクロメートル。NAから計算することも可能だが、ここでは直接設定。

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    #functions relating a BM
    # dynamic viscosity of water
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    # variance of the BM
    function sigma(T)
        sigma = sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
        return sigma
    end
    # friction coefficient
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    # generate random numbers
    function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end
    # --- LGビーム関連関数 (ユーザー提供のコードを参考に、勾配力計算用に調整) ---

    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function w_z(z, w0, zR)
        return w0 * sqrt(1 + (z/zR)^2)
    end

    """
        Laguerre_L(p::Int, l::Int, x::Real)

    SpecialPolynomials.jl を使用して、関連ラゲール多項式 L_p^(l)(x) を評価します。
    """
    function Laguerre_L(p::Int, l::Int, x::Real)
        # pとlはmainから渡されるので、lが負になる可能性はないが、念のためabs(l)を使用
        l_non_negative = abs(l)
        if p < 0 || l_non_negative < 0
            throw(ArgumentError("指数 p および |l| は非負の整数でなければなりません。"))
        end
        # SpecialPolynomials.jl の Laguerre{α} は L_n^(α)(x) を計算する
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    # LGビームの強度プロファイル (正規化された形)
    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w_z(z, w0, zR)
        x_arg = 2 * r^2 / current_wz^2
        l_abs = abs(l)

        # r=0でl > 0の場合の特異点処理 (強度0)
        if r == 0 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        # (w0/current_wz)^2 はビームの伝搬による強度変化を表す
        return (w0/current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    # 粒子の分極率 (Rayleigh近似)
    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2 # 媒質の誘電率 (mu_r = 1 と仮定)
        return 4 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2 * nm^2))
    end

    # LGビームによる光勾配力計算
    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        # デカルト座標から円筒座標へ変換
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos) # atan2(y,x) がより堅牢だが、ここでは atan で十分

        # 粒子の分極率
        alpha_particle = particle_polarizability(r_p, np, nm)

        # 数値微分用の微小ステップ
        dr_step = 1e-10 # 半径方向の微分ステップ

        # 現在位置での正規化された強度
        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)

        # 半径方向の強度勾配 (dI/dr) を計算
        r_plus_dr = r + dr_step
        if r_plus_dr < 0 
            r_plus_dr = 0.0
        end
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        # レーザーパワーによる強度スケーリングファクター
        # P = I_max * (pi * w0^2 / 2) の関係から、I_max ≈ 2*P / (pi * w0^2) を使用 [ガウスビームと比べておおよそのスケーリング]
        # LGビームの全パワー P は、正規化された強度の積分 (I_norm の積分) に依存するが、
        # 簡略化のため、ガウスビームのピーク強度に相当するスケーリングを使用
        I_scaling_factor = ls_pow / (pi * w0_beam^2) 

        # 半径方向の勾配力 (F_r = (1/2) * alpha * dI/dr * (2 * I_scaling_factor))
        # 係数は理論的な正規化に依存するが、ここでは簡略化された形を採用
        F_r = alpha_particle * dI_dr * I_scaling_factor

        # 円筒座標の力をデカルト座標に変換
        # 理想的なLGビームでは方位角方向の勾配力はゼロ (F_phi = 0)
        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        Fz = 0.0 # 2Dシミュレーションのため、軸方向の力は考慮しない

        return Fx, Fy, Fz
    end

    # generate displacements of BM
    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab.* xr
        deltayb = sigmab.* yr
        return deltaxb, deltayb
    end

    # calculate a particle behavior
    # LGビームパラメータを引数に追加
    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        x_i = zeros((n+1)) #particle x-position
        y_i = zeros((n+1)) #particle y-position
        x_i[1] = x0[j, 1]#set a initial x-position
        y_i[1] = y0[j, 1]#set a initial y-position
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)

        #Let's calculate a particle behavior
        for i in 1:n
            # LGビームによる勾配力を計算
            Fx_grad, Fy_grad, Fz_grad = LG_Gradient_Force(x_i[i], y_i[i], 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            # オイラー法で位置を更新
            # ブラウン運動の変位 + 光勾配力による変位
            x_i[i+1] = x_i[i] + xb[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end

    # define parallel calculations
    # LGビームパラメータを引数に追加
    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx.* 1e6
            resulty = resulty.* 1e6
        end
        
        # ファイル名を修正し、p_idxとl_idxを含めるように変更
        # ls_pow (W) を mW に、r_p (m) を nm に変換してファイル名に含める
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm"
        
        cd("$(save_path)")
        writedlm("x_$(filename_base).txt", resultx, "\t")
        writedlm("y_$(filename_base).txt", resulty, "\t")
    end

    #main
    # LGビームパラメータを引数に追加
    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx, l_idx, w_l)
        # generate displacements of Brrownian motion
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        # generate initial positionas
        # LGビームの中心は(0,0)なので、初期位置はビーム中心からの相対位置となる
        x0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        y0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# Main execution call
# p_indexとl_indexをmain関数に渡す
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了
rmprocs(workers())

 14.153252 seconds (371.70 k allocations: 18.820 MiB, 2.04% compilation time)


Task (done) @0x000001e391c41f10

In [4]:
#任意変更可能
#単一の CW レーザーを照射 ver.
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # Laguerre多項式のために追加

# --- ユーザーが変更するLGビームパラメーター ---
p_index = 1 # 半径方向モード数 (p >= 0)。ドーナツの輪の数に関係
l_index = 2 # 方位角モード数 (l は整数, l!=0 でドーナツ型)。軌道角運動量に関係
# ------------------------------------------

core_num = 20 #number of cores used for multiprocessing
addprocs(core_num)
output = "um" #If you want to output calculation results micro meters, write um here.
save_path = "." # !! ファイル保存エラーを防ぐため、現在のディレクトリに変更 !!

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials # Laguerre多項式のために追加
    using DelimitedFiles # writedlmを使用するために追加

    #parameters
    Tave = 298 #room temperature
    r_p = 100e-9 #radius of particle
    np = 1.44 #1.58 or 1.44 refractive index of particles
    nm = 1.33 #refractive index of midium
    ls_pow = 10e-3 #incident laser power(mW)
    w_l = 1064e-9# wavelength of laser
    NA = 0.9 #Numerical Aperture of objective lens
    p_num = 2 #number of particles
    t = 60 #calculate time
    n = 10 #steps
    ran = 20 #range of initial positions of pariticles(um)
    deltat = t / n #time width
    seed_num = 1 #seed value
    w0_beam = 1.0e-6 # ビームウェスト w0 (m), 例: 1マイクロメートル。

    #constant
    N_A = 6.022 * 1e23 #Avogadro constant
    R = 8.314 #gas constant
    kb = R / N_A #Boltzman constant
    c = 3e8 #light speed

    #define functions
    function eta(T)
        eta = 2.761 * exp(1713 / T) * 10 ^ (-6)
        return eta
    end
    function sigma(T)
        sigma = sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
        return sigma
    end
    function gamma(T, a)
        gamma = 6 * pi *eta(T) * a
        return gamma
    end
    function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end

    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function w_z(z, w0, zR)
        return w0 * sqrt(1 + (z/zR)^2)
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        if p < 0 || l_non_negative < 0
            throw(ArgumentError("指数 p および |l| は非負の整数でなければなりません。"))
        end
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w_z(z, w0, zR)
        x_arg = 2 * r^2 / current_wz^2
        l_abs = abs(l)

        if r == 0 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0/current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2 * nm^2))
    end

    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        dr_step = 1e-10

        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)

        r_plus_dr = r + dr_step
        if r_plus_dr < 0 
            r_plus_dr = 0.0
        end
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        I_scaling_factor = ls_pow / (pi * w0_beam^2) 

        F_r = alpha_particle * dI_dr * I_scaling_factor

        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        Fz = 0.0

        return Fx, Fy, Fz
    end

    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab.* xr
        deltayb = sigmab.* yr
        return deltaxb, deltayb
    end

    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        x_i = zeros((n+1))
        y_i = zeros((n+1))
        x_i[1] = x0[j, 1]
        y_i[1] = y0[j, 1]
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)

        for i in 1:n
            Fx_grad, Fy_grad, Fz_grad = LG_Gradient_Force(x_i[i], y_i[i], 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            x_i[i+1] = x_i[i] + xb[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end

    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        @sync @distributed for j = 1:p_num
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        end

        if output == "um"
            resultx = resultx.* 1e6
            resulty = resulty.* 1e6
        end
        
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm"
        
        # cd() は削除し、joinpathで相対パスとしてファイルを保存します
        x_filepath = joinpath(save_path, "x_$(filename_base).txt")
        y_filepath = joinpath(save_path, "y_$(filename_base).txt")
        
        writedlm(x_filepath, resultx, "\t")
        writedlm(y_filepath, resulty, "\t")
    end

    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx, l_idx, w_l)
        Random.seed!(seed_num)
        xb, yb = BM(p_num, Tave)
        x0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        y0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
    end
end

# Main execution call
@time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_index, l_index, w_l)

# プロセスを終了

 14.095834 seconds (358.54 k allocations: 17.922 MiB, 2.17% compilation time)


In [5]:
#全通り実行
#単一の CW レーザーを照射 ver.
using DelimitedFiles
using SharedArrays
using Random
using Distributed
using SpecialPolynomials # Laguerre多項式のために追加

# --- 環境設定 ---
core_num = 20 # number of cores used for multiprocessing
addprocs(core_num)

output = "um" # If you want to output calculation results micro meters, write um here.
# ファイル保存エラーを防ぐため、現在の作業ディレクトリに設定
save_path = "." 

# シミュレーションしたいモード数の範囲 (0から3まで)
p_range = 0:3
l_range = 0:3
total_runs = length(p_range) * length(l_range)
run_counter = 0

# --- グローバルパラメーター ---
Tave = 298 # room temperature
r_p = 100e-9 # radius of particle
np = 1.44 # refractive index of particles
nm = 1.33 # refractive index of midium
ls_pow = 10e-3 # incident laser power(mW)
w_l = 1064e-9 # wavelength of laser
NA = 0.9 # Numerical Aperture of objective lens
p_num = 2 # number of particles
t = 60 # calculate time
n = 10 # steps
ran = 20 # range of initial positions of pariticles(um)
deltat = t / n # time width
seed_num = 1 # seed value
w0_beam = 1.0e-6 # ビームウェスト w0 (m)

# ----------------------------------------------------------------

@everywhere begin
    using SharedArrays
    using Distributed
    using SpecialPolynomials 
    using DelimitedFiles 

    # --- 定数 ---
    N_A = 6.022 * 1e23 
    R = 8.314 
    kb = R / N_A 
    c = 3e8 

    # --- パラメータと依存関数 ---
    function eta(T)
        return 2.761 * exp(1713 / T) * 10 ^ (-6)
    end
    function sigma(T)
        # グローバルな r_p, deltat を使用
        return sqrt((2 * kb * T * deltat) / (6 * pi * eta(T) * r_p))
    end
    function gamma(T, a)
        return 6 * pi *eta(T) * a
    end
    function box_muller(u_1, u_2)
        X = sqrt.(-2 .* log.(u_1)) .* cos.(2 * pi .* u_2)
        Y = sqrt.(-2 .* log.(u_1)) .* sin.(2 * pi .* u_2)
        return X, Y
    end

    # --- LGビーム関連関数 ---
    function Rayleigh_Range(w0, lambda)
        return pi * w0^2 / lambda
    end

    function w_z(z, w0, zR)
        return w0 * sqrt(1 + (z/zR)^2)
    end

    function Laguerre_L(p::Int, l::Int, x::Real)
        l_non_negative = abs(l)
        L_poly = SpecialPolynomials.basis(SpecialPolynomials.Laguerre{Float64(l_non_negative)}, p)
        return L_poly(x)
    end

    function LG_Spatial_Intensity_Normalized(r, z, p, l, w0, lambda)
        zR = Rayleigh_Range(w0, lambda)
        current_wz = w_z(z, w0, zR)
        x_arg = 2 * r^2 / current_wz^2
        l_abs = abs(l)

        if r == 0 && l_abs != 0
            return 0.0
        end

        radial_term_power = x_arg^l_abs
        laguerre_term_squared = (Laguerre_L(p, l, x_arg))^2

        return (w0/current_wz)^2 * radial_term_power * laguerre_term_squared * exp(-x_arg)
    end

    function particle_polarizability(a, np, nm)
        epsilon_m = nm^2
        return 4 * pi * epsilon_m * a^3 * ((np^2 - nm^2) / (np^2 + 2 * nm^2))
    end

    function LG_Gradient_Force(x_pos, y_pos, z_pos, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)
        r = sqrt(x_pos^2 + y_pos^2)
        phi = atan(y_pos, x_pos)
        alpha_particle = particle_polarizability(r_p, np, nm)
        dr_step = 1e-10

        I_current = LG_Spatial_Intensity_Normalized(r, z_pos, p_idx, l_idx, w0_beam, w_l)

        r_plus_dr = r + dr_step
        if r_plus_dr < 0 
            r_plus_dr = 0.0
        end
        I_r_plus_dr = LG_Spatial_Intensity_Normalized(r_plus_dr, z_pos, p_idx, l_idx, w0_beam, w_l)
        dI_dr = (I_r_plus_dr - I_current) / dr_step

        I_scaling_factor = ls_pow / (pi * w0_beam^2) 

        F_r = alpha_particle * dI_dr * I_scaling_factor

        Fx = F_r * cos(phi)
        Fy = F_r * sin(phi)
        Fz = 0.0

        return Fx, Fy, Fz
    end

    function BM(p_num, T)
        xr = SharedArray{Float64}(p_num, n)
        yr = SharedArray{Float64}(p_num, n)
        @sync @distributed for i in 1:n
            u1 = rand(p_num, 1)
            u2 = rand(p_num, 1)
            X, Y = box_muller(u1, u2)
            xr[:, i] = X[:, 1]
            yr[:, i] = Y[:, 1]
        end
        sigmab = sigma(T)
        deltaxb = sigmab.* xr
        deltayb = sigmab.* yr
        return deltaxb, deltayb
    end

    function euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l, deltat)
        x_i = zeros((n+1))
        y_i = zeros((n+1))
        x_i[1] = x0[j, 1]
        y_i[1] = y0[j, 1]
        xb = deltaxb[j, :]
        yb = deltayb[j, :]
        gammaf = gamma(Tave, r_p)

        for i in 1:n
            Fx_grad, Fy_grad, Fz_grad = LG_Gradient_Force(x_i[i], y_i[i], 0.0, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l)

            x_i[i+1] = x_i[i] + xb[i] + Fx_grad * deltat / gammaf
            y_i[i+1] = y_i[i] + yb[i] + Fy_grad * deltat / gammaf
        end
        return x_i, y_i
    end

    function euler_para(p_num, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l, save_path, output, deltat)
        resultx = SharedArray{Float64}(p_num, n+1)
        resulty = SharedArray{Float64}(p_num, n+1)
        
        @sync @distributed for j = 1:p_num
            resultx[j, :], resulty[j, :] = euler(j, n, x0, y0, deltaxb, deltayb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l, deltat)
        end

        if output == "um"
            resultx = resultx.* 1e6
            resulty = resulty.* 1e6
        end
        
        # ファイル名を生成: LG_pX_lY_...nm.txt
        power_mW = Int(round(ls_pow * 1000))
        radius_nm = Int(round(r_p * 1e9))
        filename_base = "LG_p$(p_idx)_l$(l_idx)_$(power_mW)mW_$(radius_nm)nm"
        
        x_filepath = joinpath(save_path, "x_$(filename_base).txt")
        y_filepath = joinpath(save_path, "y_$(filename_base).txt")
        
        writedlm(x_filepath, resultx, "\t")
        writedlm(y_filepath, resulty, "\t")
    end

    function main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx, l_idx, w_l, save_path, output, deltat, seed_num)
        Random.seed!(seed_num)
        
        xb, yb = BM(p_num, Tave) 
        
        x0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        y0 = rand(p_num, 1).* 2 * -ran * 1e-6.+ ran * 1e-6
        
        euler_para(p_num, n, x0, y0, xb, yb, Tave, r_p, ls_pow, w0_beam, np, nm, p_idx, l_idx, w_l, save_path, output, deltat)
    end
end

# --- メイン実行: 全モードの組み合わせでシミュレーションを実行 ---
println("--- LGビーム全モードシミュレーション開始 (p=0-3, l=0-3, 全$(total_runs)通り) ---")
for p_idx in p_range
    for l_idx in l_range
        global run_counter += 1
        println("$(run_counter)/$(total_runs) 実行中: p_index = $(p_idx), l_index = $(l_idx)")
        
        # main関数を呼び出し、必要な引数を全て渡す
        @time main(p_num, n, Tave, r_p, ls_pow, w0_beam, np, nm, ran, p_idx, l_idx, w_l, save_path, output, deltat, seed_num)
        
        println("p=$(p_idx), l=$(l_idx) の結果がファイルに保存されました。")
    end
end

# プロセスを終了
println("--- 全てのシミュレーション完了。プロセスを終了します ---")

--- LGビーム全モードシミュレーション開始 (p=0-3, l=0-3, 全16通り) ---
1/16 実行中: p_index = 0, l_index = 0
 13.277880 seconds (365.40 k allocations: 18.339 MiB, 2.26% compilation time)
p=0, l=0 の結果がファイルに保存されました。
2/16 実行中: p_index = 0, l_index = 1
  2.401783 seconds (23.78 k allocations: 1.415 MiB, 0.12% compilation time)
p=0, l=1 の結果がファイルに保存されました。
3/16 実行中: p_index = 0, l_index = 2
  1.708682 seconds (28.66 k allocations: 1.940 MiB, 1.20% gc time)
p=0, l=2 の結果がファイルに保存されました。
4/16 実行中: p_index = 0, l_index = 3
  0.756741 seconds (18.67 k allocations: 1.157 MiB)
p=0, l=3 の結果がファイルに保存されました。
5/16 実行中: p_index = 1, l_index = 0
  0.506481 seconds (18.23 k allocations: 1.134 MiB)
p=1, l=0 の結果がファイルに保存されました。
6/16 実行中: p_index = 1, l_index = 1
  0.742474 seconds (18.83 k allocations: 1.178 MiB)
p=1, l=1 の結果がファイルに保存されました。
7/16 実行中: p_index = 1, l_index = 2
  0.509892 seconds (18.23 k allocations: 1.131 MiB)
p=1, l=2 の結果がファイルに保存されました。
8/16 実行中: p_index = 1, l_index = 3
  0.370208 seconds (17.79 k allocations: 1.111 MiB)
